<a href="https://colab.research.google.com/github.com/sfuller14/DeBERTa_Entity-Focused_Fine-Tuning/blob/master/DeBERTa_Entity-Focused_Fine-Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Check

In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!nvidia-smi -L
!nvidia-smi

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-e6ae7987-7758-867b-238e-bda7cbe3c6e6)
Tue Mar  7 07:36:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    49W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
          

# Imports

In [5]:
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ColabCode/MLE/DeBERTa_Entity-Focused_Fine-Tuning

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

!pip install -q transformers[sentencepiece]
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModel, AutoConfig, TFAutoModelForSequenceClassification, pipeline, TFDebertaV2Model
!pip install -q datasets
from datasets import Dataset

from pprint import pprint
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from scipy.spatial.distance import cosine


import warnings
warnings.filterwarnings("ignore")
import keras.backend as K

import spacy
import nltk
import re
from nltk.probability import FreqDist
nltk.download('stopwords')
# nlp = spacy.load("en_core_web_sm")
from nltk.corpus import stopwords
import string
stop_words = set(stopwords.words('english')) | set([p for p in string.punctuation])


import datetime as dt
import pytz

import xgboost as xgb
from sklearn.svm import SVR

!pip install -q git+https://github.com/hyperopt/hyperopt-sklearn.git
from hpsklearn import HyperoptEstimator, any_regressor, any_preprocessing
from sklearn.metrics import mean_squared_error
from hyperopt import tpe

import math

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ColabCode/MLE/DeBERTa_Entity-Focused_Fine-Tuning
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.3 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 23.3 MB/s eta 0:00:00


# EDA

## Get Data

In [ ]:
# Get data from here:
# !git clone https://bitbucket.org/ssix-project/semeval-2017-task-5-subtask-2.git ./data

## Preprocessing

In [6]:
global current
current= 'microsoft/deberta-v3-base'
global checkpoint
checkpoint = current
global tokenizer
tokenizer = AutoTokenizer.from_pretrained(current, use_fast=True, add_prefix_space=True)
global config
config = AutoConfig.from_pretrained(checkpoint)
global ner_model
ner_model = pipeline('ner', 
                     model="dbmdz/bert-large-cased-finetuned-conll03-english", 
                     tokenizer=AutoTokenizer.from_pretrained("bert-base-cased"), 
                     aggregation_strategy="simple", 
                     use_fast=True)
output.clear()

### Function to replace the target entity with the token associated with the word 'Company'

In [7]:
def replace_entity_with_company(row):#, model_name='Gladiator/microsoft-deberta-v3-large_ner_conll2003', tokenizer_name='microsoft/deberta-v3-large'):

    entity_str, headline, id = row['company'], row['title'], row['id']
    
    # Get the entities in the input text
    entities = ner_model(headline)

    # Check if entity string appears exactly in headline
    if entity_str in headline:
        return headline.replace(entity_str, " Company ")

    # Check if entity string appears exactly in headline
    if entity_str.lower() in headline.lower():
        return headline.lower().replace(entity_str.lower(), " Company ")

    # Find the first entity of type "ORG"
    org_entity = None
    for ent in entities:
        if ent['entity_group'] == 'ORG':
            org_entity = ent
            break

    org_entities = [ent for ent in entities if ent['entity_group'] == "ORG"]
    
    # If there is only one entity of type "ORG", replace it with "company"
    if len(org_entities) == 1:
        start, end = org_entity['start'], org_entity['end']
        return headline[:start] + " Company " + headline[end:]
    
    # If there are multiple entities of type "ORG" and one of them starts with the same first letter of the entity string,
    # replace that "ORG" with "company"
    if len(org_entities) > 1:
        for org_entity in org_entities:
            start, end = org_entity['start'], org_entity['end']
            if headline[start] == entity_str[0]:
                return headline[:start] + " Company " + headline[end:]

    # If there is no entity of type ORG, replace first entity of any type
    if len(org_entities) > 0:
        start, end = org_entities[0]["start"], org_entities[0]["end"]
        return headline[:start] + " Company " + headline[end:]

    # Check for any entity
    if len(entities) > 1:
        for ent in entities:
            start, end = ent['start'], ent['end']
            if headline[start] == entity_str[0]:
                return headline[:start] + " Company " + headline[end:]

    # Check for any entity
    if len(entities) > 0:
        start, end = entities[0]["start"], entities[0]["end"]
        return headline[:start] + " Company " + headline[end:]

    # Check for noun
    for token in entities:
        if token.pos_ == "NOUN":
            start, end = token["start"], token["end"]
            return headline[:start] + " Company " + headline[end:]

    # Replace first word
    return " Company " + headline.split(" ", 1)[1]

In [8]:
# prefix cleaning function
def clean_prefix(df):

    df['title'] = df['title'].apply(lambda x: re.sub('\s{2,}', ' ', x))
    df['title'] = df['title'].str.strip()

    # extract the column as a Series
    title_series = df['title']

    # split the Series into words
    words = []
    for sentence in title_series:
        words += sentence.split()

    # create a frequency distribution of the words
    freq_dist = FreqDist(words)

    # sort the frequency distribution in descending order
    sorted_freq = sorted(freq_dist.items(), key=lambda x: x[1], reverse=True)
    common_starters = set([t[0] for t in sorted_freq[:50]])

    exclude = list(set(df['company'].tolist())) + list(stop_words)
    common_starters = [starter for starter in common_starters if starter not in exclude]
    common_starters
    for starter in common_starters:
        if len(df.loc[(df['title'].str.lower().str.startswith(starter)),['company', 'title']]) > 5:
            # print(starter)
            # display(df.loc[(df['title'].str.startswith(starter)),['company', 'title']].sort_values('title').head(10))
            pass

    ridx = []
    # Remove identified prefixes
    for prefix in ['industry news', 'news feed', 'ftse 100 movers: ', 
                   'update 1-', 'update 2-', 'update 3-', 'update 4-', 'update 5-', 'update 6-', 'update -', 
                   'companies', 'refile-', 'update:', 'markets']:
        if not df.loc[(df['title'].str.lower().str.startswith(prefix)),['company', 'title']].empty:
            df_filtered = df.loc[(df['title'].str.lower().str.startswith(prefix)),['company', 'title']].head(10)
            df_filtered['new_title'] = df_filtered['title'].str[len(prefix):]
            display(df_filtered[['company', 'title', 'new_title']])
            print()
        len_prefix = len(prefix)
        ridx += df.loc[(df['title'].str.lower().str.startswith(prefix)), 'id'].index.tolist()
        # Replace the prefix with an empty string
        df.loc[df['title'].str.lower().str.startswith(prefix), 'title'] = df.loc[df['title'].str.lower().str.startswith(prefix), 'title'].str[len_prefix:]

    return df

# For experiments that mask the target entity in the input, we need to create a new dataset with the masked input
def masking_func(df):

    df = clean_prefix(df)

    # apply preprocess_headline to the DataFrame
    df['title'] = df.apply(replace_entity_with_company, axis=1)

    # make the 'company' and 'title' columns lowercase
    # df['company'] = df['company'].str.lower()
    # df['title'] = df['title'].str.lower()

    # check that all rows in df['title_with_entity_masked'] contain the string ' company ' using a unit test
    # and print 'All headlines successfully masked with "company"' if the test passes
    assert all(' Company ' in s for s in df['title'])
    print('All headlines successfully masked with " Company "')
    df['title'] = df['title'].apply(lambda x: re.sub('\s{2,}', ' ', x))
    df['title'] =  df['title'].str.strip()

    return Dataset.from_pandas(df)

### Apply the above functions

In [9]:
# Read in data
raw_train_df = pd.read_json("data/Headline_Trainingdata.json")
raw_val_df = pd.read_json("data/Headline_Validationdata.json")
raw_test_df = pd.read_json("data/Headlines_Testdata_withscores_clean.json")
raw_all = pd.concat([raw_train_df, raw_val_df, raw_test_df])

# Create a new huggingface Dataset for the unmasked headlines
raw_train_ds, raw_val_ds, raw_test_ds = Dataset.from_pandas(clean_prefix(raw_train_df)), Dataset.from_pandas(clean_prefix(raw_val_df)), Dataset.from_pandas(clean_prefix(raw_test_df))
raw_datasets = {"train": raw_train_ds, "val": raw_val_ds, "test": raw_test_ds}

# Create a new huggingface Dataset for the masked headlines
masked_train_ds, masked_val_ds, masked_test_ds = masking_func(raw_train_df), masking_func(raw_val_df), masking_func(raw_test_df)
masked_datasets = {"train": masked_train_ds, "val": masked_val_ds, "test": masked_test_ds}

masked_all = pd.concat([raw_train_df, raw_val_df, raw_test_df])

,company,title,new_title
92,Wolseley,Industry NewsWolseley confident in reslilience amid mixed markets,Wolseley confident in reslilience amid mixed markets
309,Weir Group,"Industry NewsRevenue, earnings take a tumble at Weir Group","Revenue, earnings take a tumble at Weir Group"
426,Wood Group,Industry NewsWood Group wins multi-million dollar contract with BP,Wood Group wins multi-million dollar contract with BP
441,G4S,"Industry NewsG4S makes 'positive' start to year, no new impairments","G4S makes 'positive' start to year, no new impairments"
574,Hammerson,Industry NewsHammerson joint venture buys Dublin loan portfolio,Hammerson joint venture buys Dublin loan portfolio
614,Travis Perkins,Industry NewsStrong results at Travis Perkins marred by plumbing impairment,Strong results at Travis Perkins marred by plumbing impairment
688,Petrofac,Industry NewsPetrofac secures $250m North Sea contract,Petrofac secures $250m North Sea contract
806,BP,Industry NewsWood Group wins multi-million dollar contract with BP,Wood Group wins multi-million dollar contract with BP


,company,title,new_title
77,AstraZeneca,News FeedFTSE 100 movers: Standard Chartered lifted while AstraZeneca sinks,FTSE 100 movers: Standard Chartered lifted while AstraZeneca sinks
281,Schroders,"News FeedSchroders books solid earnings growth, several board changes","Schroders books solid earnings growth, several board changes"
310,Ashtead,"News FeedFTSE 100 movers: Ashtead jumps on strong interims; Glencore, BP in ...","FTSE 100 movers: Ashtead jumps on strong interims; Glencore, BP in ..."
405,ICE,News FeedFTSE 100 movers: LSE surges as ICE says mulling offer; Ashtead and Barclays tank,FTSE 100 movers: LSE surges as ICE says mulling offer; Ashtead and Barclays tank
498,Barclays,News FeedFTSE 100 movers: LSE surges as ICE says mulling offer; Ashtead and Barclays tank,FTSE 100 movers: LSE surges as ICE says mulling offer; Ashtead and Barclays tank


,company,title,new_title
77,AstraZeneca,FTSE 100 movers: Standard Chartered lifted while AstraZeneca sinks,Standard Chartered lifted while AstraZeneca sinks
310,Ashtead,"FTSE 100 movers: Ashtead jumps on strong interims; Glencore, BP in ...","Ashtead jumps on strong interims; Glencore, BP in ..."
405,ICE,FTSE 100 movers: LSE surges as ICE says mulling offer; Ashtead and Barclays tank,LSE surges as ICE says mulling offer; Ashtead and Barclays tank
498,Barclays,FTSE 100 movers: LSE surges as ICE says mulling offer; Ashtead and Barclays tank,LSE surges as ICE says mulling offer; Ashtead and Barclays tank


,company,title,new_title
19,BP,"UPDATE 1-BP reports worst annual loss in at least 20 years, cuts more jobs","BP reports worst annual loss in at least 20 years, cuts more jobs"
32,Shire,UPDATE 1-Shire takes Q2 earnings pain for long-term gain,Shire takes Q2 earnings pain for long-term gain
36,BP,UPDATE 1-Oil major BP freezes pay in 2015 to cut costs,Oil major BP freezes pay in 2015 to cut costs
54,Shell,UPDATE 1-Seattle flotilla protests Shell's Arctic drilling plans,Seattle flotilla protests Shell's Arctic drilling plans
192,Tullow Oil,UPDATE 1-Britain's Tullow says makes new oil find in Kenya,Britain's Tullow says makes new oil find in Kenya
209,Petrofac,UPDATE 1-Petrofac posts net loss hurt by Shetland Islands project costs,Petrofac posts net loss hurt by Shetland Islands project costs
221,Royal Bank of Scotland Group plc,"UPDATE 1-Nomura, RBS must pay $806 mln in mortgage bond case-US judge","Nomura, RBS must pay $806 mln in mortgage bond case-US judge"
304,GSK,UPDATE 1-GSK-linked investigator freed early from China jail - source,GSK-linked investigator freed early from China jail - source
341,Morrisons,UPDATE 1-Dairy Crest loses a third of Morrisons milk contract,Dairy Crest loses a third of Morrisons milk contract
513,Wells Fargo,UPDATE 1-Berkshire applies to boost Wells Fargo stake above 10 pct,Berkshire applies to boost Wells Fargo stake above 10 pct


,company,title,new_title
39,EasyJet,UPDATE 2-EasyJet sees better first half on lower fuel bill,EasyJet sees better first half on lower fuel bill
132,Barclays,"UPDATE 2-Exchanges, Barclays win dismissal of US high-frequency trading case","Exchanges, Barclays win dismissal of US high-frequency trading case"
539,Prudential,"UPDATE 2-Prudential will see strong first-half earnings, outgoing CEO says","Prudential will see strong first-half earnings, outgoing CEO says"
578,SABMiller,UPDATE 2-Pricey beers lift SABMiller's quarterly underlying sales,Pricey beers lift SABMiller's quarterly underlying sales
802,MillerCoors,"UPDATE 2-AB InBev launches SAB bid, to sell MillerCoors stake","AB InBev launches SAB bid, to sell MillerCoors stake"


...

,company,title,new_title
63,Aer Lingus,"UPDATE: EasyJet Passenger Numbers, Aer Lingus Traffic Up In February","EasyJet Passenger Numbers, Aer Lingus Traffic Up In February"
124,Lehman,UPDATE: Barclays Expects Gain After Settling With Lehman Bros Trustee,Barclays Expects Gain After Settling With Lehman Bros Trustee
153,AXA,"UPDATE: CIB, Legal & General Sell Egyptian Life Joint Venture To AXA","CIB, Legal & General Sell Egyptian Life Joint Venture To AXA"
388,Barclays,UPDATE: Barclays Expects Gain After Settling With Lehman Bros Trustee,Barclays Expects Gain After Settling With Lehman Bros Trustee
442,Persimmon,"UPDATE: Persimmon Profit Up Strongly, Outlook Positive","Persimmon Profit Up Strongly, Outlook Positive"
450,British American Tobacco plc,"UPDATE: British American Tobacco 2014 Profit Drops, Difficult 2015 Ahead","British American Tobacco 2014 Profit Drops, Difficult 2015 Ahead"


,company,title,new_title
95,Baxalta,MarketsShire up 2.5% and Baxalta up 6% on $32bn deal,Shire up 2.5% and Baxalta up 6% on $32bn deal
158,Wolseley,MarketsWolseley shares wilt 8.8% after full year results,Wolseley shares wilt 8.8% after full year results



All headlines successfully masked with " Company "
All headlines successfully masked with " Company "


,company,title,new_title
156,Lloyds Banking Group PLC,UPDATE 4-Britain's Lloyds to pay first dividend since rescue,Britain's Lloyds to pay first dividend since rescue



All headlines successfully masked with " Company "


In [12]:
del ner_model

**Raw datasets**

In [ ]:
raw_all.head()

,id,company,title,sentiment
0,2,Morrisons,Morrisons book second consecutive quarter of sales growth,0.430
1,3,IMI,IMI posts drop in first-quarter organic revenue; warns on full year,-0.344
2,4,Glencore,"Glencore to refinance its short-term debt early, shares rise",0.340
3,5,Ryanair,EasyJet attracts more passengers in June but still lags Ryanair,0.259
4,6,Barclays,Barclays 'bad bank' chief to step down,-0.231


In [ ]:
raw_datasets

{'train': Dataset({
     features: ['id', 'company', 'title', 'sentiment'],
     num_rows: 914
 }), 'val': Dataset({
     features: ['id', 'company', 'title', 'sentiment'],
     num_rows: 228
 }), 'test': Dataset({
     features: ['id', 'company', 'title', 'sentiment'],
     num_rows: 491
 })}

**Masked datasets**



In [ ]:
masked_all.head()

,id,company,title,sentiment
0,2,Company,Company book second consecutive quarter of sales growth,0.430
1,3,Company,Company posts drop in first-quarter organic revenue; warns on full year,-0.344
2,4,Company,"Company to refinance its short-term debt early, shares rise",0.340
3,5,Company,EasyJet attracts more passengers in June but still lags Company,0.259
4,6,Company,Company 'bad bank' chief to step down,-0.231


In [ ]:
masked_datasets

{'train': Dataset({
     features: ['id', 'company', 'title', 'sentiment'],
     num_rows: 914
 }), 'val': Dataset({
     features: ['id', 'company', 'title', 'sentiment'],
     num_rows: 228
 }), 'test': Dataset({
     features: ['id', 'company', 'title', 'sentiment'],
     num_rows: 491
 })}

# Data Prep & Model Training Functions

## Tokenize datasets

### Function to convert from HF dataset to tfds
Conditional arg to specify if 'company' token should be inserted

In [14]:
def create_tokenized_tf_datasets(ds, masked=False, batch_size=32):

    untokenized_datasets_dict = ds

    # tokenizer = AutoTokenizer.from_pretrained(current)

    def get_max_tokenized_length(ds):
        max_len_raw = 0
        max_len_token_vector = 0

        for text_input in ds:

            if len(text_input.split(" ")) > max_len_raw:
                max_len_raw = len(text_input.split(" "))

            if len(tokenizer(text_input)['input_ids']) > max_len_token_vector:
                max_len_token_vector = len(tokenizer(text_input)['input_ids'])

        print(f'"{current}":\n\t"max_len_raw": {max_len_raw},\n\t"max_len_tokenized": {max_len_token_vector}')

        return max_len_raw, max_len_token_vector

    # Get max_len_token_vector for passing into 
    max_len_raw, max_len_token_vector = get_max_tokenized_length(ds=untokenized_datasets_dict['train']['title'] + untokenized_datasets_dict['val']['title'] + untokenized_datasets_dict['test']['title'])

    def preprocess_function(examples):

        # If 'company' has been inserted in place of target entity
        if masked:
            label = examples["sentiment"] 
            headline=examples["title"]
            examples = tokenizer(examples["title"], truncation=True, padding="max_length", max_length=max_len_token_vector)
            input_ids = examples['input_ids']
            # Change this to real number
            examples["label"] = float(label)

            # get entity_idx
            entity_encoding_int = tokenizer('Company')['input_ids'][1]#tokenizer.convert_tokens_to_ids('company')
            try:
                entity_idx = input_ids.index(entity_encoding_int)
            except:
                print(headline)
                print(input_ids)
                print(examples)
            if not  input_ids[entity_idx] == entity_encoding_int:
                raise ValueError('COMPANY NOT AT EXPECTED INDEX...BREAKING')
            else:
                examples["entity_idx"] = int(entity_idx)

            return examples
        
        # If 'company' has NOT been inserted in place of target entity
        else:
            label = examples["sentiment"] 
            examples = tokenizer(examples["title"], truncation=True, padding="max_length", max_length=max_len_token_vector)
            
            # Change this to real number
            examples["label"] = float(label)
            return examples

    # Tokenize the datasets
    tokenized_datasets = dict()
    for split in untokenized_datasets_dict:
        tokenized_datasets[split] = untokenized_datasets_dict[split].map(preprocess_function)

     
    # Create tf datasets for input into the model
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

    tf_datasets_dict = dict()
    
    tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
        columns=["attention_mask", "input_ids", "entity_idx"] if masked else ['attention_mask', 'input_ids'],
        label_cols=["labels"],
        shuffle=False,
        collate_fn=data_collator,
        batch_size=batch_size,
    )

    tf_val_dataset = tokenized_datasets["val"].to_tf_dataset(
        columns=["attention_mask", "input_ids", "entity_idx"] if masked else ['attention_mask', 'input_ids'],
        label_cols=["labels"],
        shuffle=False,
        collate_fn=data_collator,
        batch_size=batch_size,
    )    

    tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
        columns=["attention_mask", "input_ids", "entity_idx"] if masked else ['attention_mask', 'input_ids'],
        label_cols=["labels"],
        shuffle=False,
        collate_fn=data_collator,
        batch_size=batch_size,
    )

    return max_len_token_vector, tokenized_datasets, tf_train_dataset, tf_val_dataset, tf_test_dataset

## Create function to fit model

In [15]:
# For model evaluation:
def cosine_score(predicted_values, true_values):
    return 1 - cosine(predicted_values, true_values)


# for training
def adj_keras_cosine_proximity(y_true, y_pred):

    u = y_pred
    v = y_true
    
    uv = K.mean(u * v)
    uu = K.mean(K.square(u))
    vv = K.mean(K.square(v))
    dist = K.abs(1.0 - uv / K.sqrt(uu * vv))

    return 1- K.clip(dist, 0.0, 2.0)

# Register the custom metric function
tf.keras.utils.get_custom_objects()['adj_keras_cosine_proximity'] = adj_keras_cosine_proximity

In [16]:
global results_dict
results_dict = {}

In [17]:
def reshape_entity(x, y):
    input_ids = x["input_ids"]
    attention_mask = x["attention_mask"]
    entity_idx = tf.reshape(x["entity_idx"], [-1, 1])
    return (input_ids, attention_mask, entity_idx), y
    
def create_tokenized_tf_datasets_caller(ds_type, batch_size):
    if ds_type == "masked_extra_field":
        max_seq, datasets_dict, train_ds, val_ds, test_ds = create_tokenized_tf_datasets(ds = masked_datasets, masked=True, batch_size=batch_size)
        train_ds = train_ds.map(reshape_entity)
        val_ds = val_ds.map(reshape_entity)
        test_ds = test_ds.map(reshape_entity)
    elif ds_type == "masked":
        max_seq, datasets_dict, train_ds, val_ds, test_ds = create_tokenized_tf_datasets(ds = masked_datasets, masked=False, batch_size=batch_size)
    elif ds_type == "raw":
        max_seq, datasets_dict, train_ds, val_ds, test_ds = create_tokenized_tf_datasets(ds = raw_datasets, masked=False, batch_size=batch_size)
    return max_seq, datasets_dict, train_ds, val_ds, test_ds

def fit_model(name, model_fn, train_ds, val_ds):
    results = []
    histories = []
    best_metric = 0
    for i in range(5):
        model = model_fn()
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss="huber_loss", metrics=['adj_keras_cosine_proximity'])
        result, history, model = train_and_evaluate(model, train_ds, val_ds)
        if result > best_metric:
            best_metric = result
            best_model = model
            best_model.save(f'/content/drive/MyDrive/ColabCode/MLE/best_models/{name}.h5')
        results.append(result)
        histories.append(history)

    print(f'{name} -- Validation set results:\n')
    print(f'cosine_similarities = {results}')
    print(f'mean_cosine_similarities = {np.median(results)}')
    print(f'best_cosine_similarity = {best_metric}')

    results_dict[name] = [np.median(results), results, histories, best_metric]

def train_and_evaluate(model, train_ds, val_ds):
    early_stopping_monitor = EarlyStopping(patience=6, restore_best_weights=True, monitor="val_adj_keras_cosine_proximity", mode="max")
    history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[early_stopping_monitor])
    
    predicted_sentiments = model.predict(val_ds)

    result = cosine_score(predicted_sentiments.flatten(), np.concatenate([y for x, y in val_ds], axis=0))

    print(f'cosine similarity on validation set: {result}')
    return result, history, model

In [18]:
from typing import Dict, Optional, Tuple, Union, List

def shape_list(tensor: Union[tf.Tensor, np.ndarray]) -> List[int]:
    """
    Deal with dynamic shape in tensorflow cleanly.
    Args:
        tensor (`tf.Tensor` or `np.ndarray`): The tensor we want the shape of.
    Returns:
        `List[int]`: The shape of the tensor as a list.
    """
    if isinstance(tensor, np.ndarray):
        return list(tensor.shape)

    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

class TFDebertaV2StableDropout(tf.keras.layers.Layer):
    """
    Optimized dropout module for stabilizing the training
    Args:
        drop_prob (float): the dropout probabilities
    """

    def __init__(self, drop_prob, **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prob

    @tf.custom_gradient
    def xdropout(self, inputs):
        """
        Applies dropout to the inputs, as vanilla dropout, but also scales the remaining elements up by 1/drop_prob.
        """
        mask = tf.cast(
            1
            - tf.compat.v1.distributions.Bernoulli(probs=1.0 - self.drop_prob).sample(sample_shape=shape_list(inputs)),
            tf.bool,
        )
        scale = tf.convert_to_tensor(1.0 / (1 - self.drop_prob), dtype=tf.float32)
        if self.drop_prob > 0:
            inputs = tf.where(mask, 0.0, inputs) * scale

        def grad(upstream):
            if self.drop_prob > 0:
                return tf.where(mask, 0.0, upstream) * scale
            else:
                return upstream

        return inputs, grad

    def call(self, inputs: tf.Tensor, training: tf.Tensor = False):
        if training:
            return self.xdropout(inputs)
        return inputs

# Experiments
Model selection using validation set

## Baselines 

### [CLS] Only - No Replacement
##### This is directly equivalent to TFDebertaForSequenceClassification
- No target entity replacement
- [CLS] token only
  - Pooled and run through dense layer

In [ ]:
def cls_baseline():
    input_ids_layer = tf.keras.layers.Input(shape=(max_seq,), dtype=tf.int32, name='input_ids')
    attention_mask_layer = tf.keras.layers.Input((max_seq,), dtype=tf.int32, name='attention_mask')


    bert_layer = TFAutoModel.from_pretrained(checkpoint, output_hidden_states=True, num_labels=1)
    bert_outputs = bert_layer(input_ids_layer, attention_mask=attention_mask_layer)
    hidden_states = bert_outputs.last_hidden_state
    cls_token_LHS = hidden_states[:, 0, :]

    pooler_layer = tf.keras.layers.Dense(units=768, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="tanh",
                                        name="pooler")(cls_token_LHS)

    cls_dropout = TFDebertaV2StableDropout(drop_prob=0.1, name='cls_dropout')(pooler_layer)
    classifier = tf.keras.layers.Dense(units=1,
                                    kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                    name="classifier")(cls_dropout)

    model = tf.keras.models.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=classifier)
    return model

In [ ]:
name = "cls_baseline"
ds_type="raw"
max_seq, datasets_dict, train_ds, val_ds, _ = create_tokenized_tf_datasets_caller(ds_type, batch_size=32)
fit_model(name, cls_baseline, train_ds, val_ds)

### [CLS] Only - With Replacement
- Target entity replaced with 'company' (i.e. same architecture as above; different dataset)
- [CLS] token only
  - Pooled and run through dense layer

In [ ]:
def cls_tgt_replaced_baseline():
    input_ids_layer = tf.keras.layers.Input(shape=(max_seq,), dtype=tf.int32, name='input_ids')
    attention_mask_layer = tf.keras.layers.Input((max_seq,), dtype=tf.int32, name='attention_mask')


    bert_layer = TFAutoModel.from_pretrained(checkpoint, output_hidden_states=True, num_labels=1)
    bert_outputs = bert_layer(input_ids_layer, attention_mask=attention_mask_layer)
    hidden_states = bert_outputs.last_hidden_state # shape = [914, 29, 768]
    cls_token_LHS = hidden_states[:, 0, :] # shape = [914, 768]

    pooler_layer = tf.keras.layers.Dense(units=768, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="tanh",
                                        name="pooler")(cls_token_LHS)

    # config.pooler_dropout = 0
    cls_dropout = TFDebertaV2StableDropout(drop_prob=0.1, name='cls_dropout')(pooler_layer)
    classifier = tf.keras.layers.Dense(units=1,
                                    kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                    name="classifier")(cls_dropout)


    model = tf.keras.models.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=classifier)
    return model

In [ ]:
name = "cls_tgt_replaced_baseline"
ds_type="masked"
max_seq, datasets_dict, train_ds, val_ds, _ = create_tokenized_tf_datasets_caller(ds_type, batch_size=32)
fit_model(name, cls_tgt_replaced_baseline, train_ds, val_ds)

### Entity Only - With Replacement

In [ ]:
def entity_baseline():
    input_ids_layer = tf.keras.layers.Input(shape=(max_seq,), dtype=tf.int32, name='input_ids')
    attention_mask_layer = tf.keras.layers.Input((max_seq,), dtype=tf.int32, name='attention_mask')
    entity_idx_layer = tf.keras.layers.Input((1,), dtype=tf.int32, name='entity_idx')

    bert_layer = TFAutoModel.from_pretrained(checkpoint, output_hidden_states=True, num_labels=1)
    bert_outputs = bert_layer(input_ids_layer, attention_mask=attention_mask_layer)
    hidden_states = bert_outputs.last_hidden_state # --> shape = [914, 29, 768]
    # cls_token_LHS = hidden_states[:, 0, :] # --> i want this, but done dynamically
    entity_hidden_state = tf.gather(hidden_states, entity_idx_layer, batch_dims=1) # --> shape = [batch_size, 1, hidden_dim] shape = [914, 768]
    entity_hidden_state = tf.squeeze(entity_hidden_state, axis=1) # --> shape = [batch_size, hidden_dim]

    pooler_layer = tf.keras.layers.Dense(units=768, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="tanh",
                                        name="dense")(entity_hidden_state)

    cls_dropout = TFDebertaV2StableDropout(drop_prob=0.1, name='cls_dropout')(pooler_layer)
    classifier = tf.keras.layers.Dense(units=1,
                                    kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                    name="classifier")(cls_dropout)
    
    model = tf.keras.models.Model(inputs=[input_ids_layer, attention_mask_layer, entity_idx_layer], outputs=classifier)
    return model

In [ ]:
name = "entity_baseline"
ds_type="masked_extra_field"
max_seq, datasets_dict, train_ds, val_ds, _ = create_tokenized_tf_datasets_caller(ds_type, batch_size=32)
fit_model(name, entity_baseline, train_ds, val_ds)

## Attached Classification Head

### [CLS] & Entity - Position-wise FFN

In [ ]:
def ATTACHED_both_FFN_concat():
    input_ids_layer = tf.keras.layers.Input(shape=(max_seq,), dtype=tf.int32, name='input_ids_layer')
    attention_mask_layer = tf.keras.layers.Input((max_seq,), dtype=tf.int32, name='attention_mask_layer')
    entity_idx_layer = tf.keras.layers.Input((1,), dtype=tf.int32, name='entity_idx_layer')


    bert_layer = TFAutoModel.from_pretrained(checkpoint, output_hidden_states=True, num_labels=1)
    bert_outputs = bert_layer(input_ids_layer, attention_mask=attention_mask_layer)
    hidden_states = bert_outputs.last_hidden_state # --> shape = [914, 29, 768]
    entity_hidden_state = tf.gather(hidden_states, entity_idx_layer, batch_dims=1) # --> shape = [batch_size, 1, hidden_dim] shape = [914, 768]
    entity_hidden_state = tf.squeeze(entity_hidden_state, axis=1) # --> shape = [batch_size, hidden_dim]
    cls_token_LHS = hidden_states[:, 0, :] 

    ff_entity1 = tf.keras.layers.Dense(units=3072, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="gelu",
                                        name="ff_hidden1")(entity_hidden_state)
    ff_entity2 = tf.keras.layers.Dense(units=768, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="gelu",
                                        name="ff_hidden2")(ff_entity1)


    ff_cls1 = tf.keras.layers.Dense(units=3072, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="gelu",
                                        name="ff_cls1")(cls_token_LHS)
    ff_cls2 = tf.keras.layers.Dense(units=768, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="gelu",
                                        name="ff_cls2")(ff_cls1)

    concat_layer = tf.keras.layers.concatenate([ff_cls2, ff_entity2], name="concat")

    pooler_layer = tf.keras.layers.Dense(units=768, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="tanh",
                                        name="dense")(concat_layer)

    cls_dropout = TFDebertaV2StableDropout(drop_prob=0.1, name='cls_dropout')(pooler_layer)
    classifier = tf.keras.layers.Dense(units=1,
                                    kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                    name="classifier")(cls_dropout)

    model = tf.keras.models.Model(inputs=[input_ids_layer, attention_mask_layer, entity_idx_layer], outputs=classifier)
    return model

In [ ]:
name = "ATTACHED_both_FFN_concat"
ds_type="masked_extra_field"
max_seq, datasets_dict, train_ds, val_ds, _ = create_tokenized_tf_datasets_caller(ds_type, batch_size=32)
fit_model(name, ATTACHED_both_FFN_concat, train_ds, val_ds)

### [CLS] & Entity - CNN BiLSTM

In [23]:
def ATTACHED_both_cnn_bilstm():
    input_ids_layer = tf.keras.layers.Input(shape=(max_seq,), dtype=tf.int32, name='input_ids_layer')
    attention_mask_layer = tf.keras.layers.Input((max_seq,), dtype=tf.int32, name='attention_mask_layer')
    entity_idx_layer = tf.keras.layers.Input((1,), dtype=tf.int32, name='entity_idx_layer')

    bert_layer = TFAutoModel.from_pretrained(checkpoint, output_hidden_states=True, num_labels=1)
    bert_outputs = bert_layer(input_ids_layer, attention_mask=attention_mask_layer)
    hidden_states = bert_outputs.last_hidden_state # --> shape = [914, 29, 768]
    entity_hidden_state = tf.gather(hidden_states, entity_idx_layer, batch_dims=1) # --> shape = [batch_size, 1, hidden_dim] shape = [914, 768]
    entity_hidden_state = tf.squeeze(entity_hidden_state, axis=1) # --> shape = [batch_size, hidden_dim]

    combined_pooled_hidden_states = tf.keras.layers.concatenate([hidden_states[:, 0, :], entity_hidden_state], axis=1)
    
    # add a CNN-BiLSTM layer
    cnn_lstm = tf.keras.Sequential()
    cnn_lstm.add(tf.keras.layers.Reshape((1, combined_pooled_hidden_states.shape[1])))
    cnn_lstm.add(tf.keras.layers.Conv1D(128, kernel_size=3, padding='same', activation='relu'))
    cnn_lstm.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
    cnn_lstm_output = cnn_lstm(combined_pooled_hidden_states)

    cls_dropout = TFDebertaV2StableDropout(drop_prob=0.1, name='cls_dropout')(cnn_lstm_output)
    classifier = tf.keras.layers.Dense(units=1,
                                    kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                    name="classifier")(cls_dropout)

    model = tf.keras.models.Model(inputs=[input_ids_layer, attention_mask_layer, entity_idx_layer], outputs=classifier)

    return model

In [ ]:
name = "ATTACHED_both_cnn_bilstm"
ds_type="masked_extra_field"
max_seq, datasets_dict, train_ds, val_ds, _ = create_tokenized_tf_datasets_caller(ds_type, batch_size=32)
fit_model(name, ATTACHED_both_cnn_bilstm, train_ds, val_ds) 

## Detached Classification Head

**Secondary models - Data Prep**

In [36]:
cls_baseline_model = keras.models.load_model(f'/content/drive/MyDrive/ColabCode/MLE/best_models/cls_baseline.h5', 
                                             custom_objects={"TFDebertaV2Model": TFDebertaV2Model, "TFDebertaV2StableDropout":TFDebertaV2StableDropout})

entity_baseline_model = keras.models.load_model(f'/content/drive/MyDrive/ColabCode/MLE/best_models/entity_baseline.h5', 
                                             custom_objects={"TFDebertaV2Model": TFDebertaV2Model, "TFDebertaV2StableDropout":TFDebertaV2StableDropout})

In [38]:
ds_type = "raw"
CLS_max_seq, CLS_datasets_dict, CLS_train_ds, CLS_val_ds, CLS_test_ds = create_tokenized_tf_datasets_caller(ds_type, batch_size=32)

ds_type = "masked_extra_field"
ENT_max_seq, ENT_datasets_dict, ENT_train_ds, ENT_val_ds, ENT_test_ds = create_tokenized_tf_datasets_caller(ds_type, batch_size=32)
output.clear()

In [39]:
# Extract the input layers from the model
CLS_input_ids_layer = cls_baseline_model.get_layer('input_ids')
CLS_attention_mask_layer = cls_baseline_model.get_layer('attention_mask')

# Extract the BERT layer from the model
CLS_bert_layer = cls_baseline_model.layers[2]

# Create a new model that outputs only the last_hidden_state variable
CLS_last_hidden_state_model = tf.keras.models.Model(inputs=[CLS_input_ids_layer.input, CLS_attention_mask_layer.input], outputs=CLS_bert_layer.output)

In [40]:
# Extract the input layers from the model
ENTITY_input_ids_layer = entity_baseline_model.get_layer('input_ids')
ENTITY_attention_mask_layer = entity_baseline_model.get_layer('attention_mask')
ENTITY_idx_layer = entity_baseline_model.get_layer('entity_idx')

# Extract the BERT layer from the model
ENTITY_bert_layer = entity_baseline_model.layers[2]

# Create a new model that outputs only the last_hidden_state variable
ENTITY_last_hidden_state_model = tf.keras.models.Model(inputs=[ENTITY_input_ids_layer.input, ENTITY_attention_mask_layer.input, ENTITY_idx_layer.input], outputs=ENTITY_bert_layer.output)#
ENTITY_idx_model = tf.keras.models.Model(inputs=[ENTITY_input_ids_layer.input, ENTITY_attention_mask_layer.input, ENTITY_idx_layer.input], outputs=ENTITY_idx_layer.output)#

In [41]:
def create_arrays(ds_name, cls_ds, ent_ds):
    # CLS
    CLS_last_hidden_state_output = CLS_last_hidden_state_model.predict(cls_ds)  # <------- HERE
    CLS_token_ouput = CLS_last_hidden_state_output.last_hidden_state[:, 0, :]

    CLS_token_ouput_np = np.vstack(CLS_token_ouput)

    # ENTITY
    ENTITY_last_hidden_state_output = ENTITY_last_hidden_state_model.predict(ent_ds)  # <------- HERE
    ENTITY_idx_layer_output = ENTITY_idx_model.predict(ent_ds)  # <------- HERE

    ENTITY_hidden_states = ENTITY_last_hidden_state_output.last_hidden_state

    ENTITY_hidden_state = tf.gather(ENTITY_hidden_states, ENTITY_idx_layer_output, batch_dims=1)
    ENTITY_hidden_state = tf.squeeze(ENTITY_hidden_state, axis=1)

    ENTITY_hidden_state_np = ENTITY_hidden_state.numpy()

    return ENTITY_hidden_state_np, CLS_token_ouput_np


# --TRAIN--
train_ENTITY_hidden_state_np, train_CLS_token_ouput_np = create_arrays('train', CLS_train_ds, ENT_train_ds)
y_train = ENT_datasets_dict['train']['label']
# convert to tensorflow dataset
secondary_train_ds = tf.data.Dataset.from_tensor_slices(({'entity_hidden_state_layer': train_ENTITY_hidden_state_np, 'cls_token_LHS_layer': train_CLS_token_ouput_np}, y_train)).batch(32)


# --VAL--
val_ENTITY_hidden_state_np, val_CLS_token_ouput_np = create_arrays('val', CLS_val_ds, ENT_val_ds)
y_val = ENT_datasets_dict['val']['label']
# convert to tensorflow dataset
secondary_val_ds = tf.data.Dataset.from_tensor_slices(({'entity_hidden_state_layer': val_ENTITY_hidden_state_np, 'cls_token_LHS_layer': val_CLS_token_ouput_np}, y_val)).batch(32)

# --TEST--
test_ENTITY_hidden_state_np, test_CLS_token_ouput_np = create_arrays('test', CLS_test_ds, ENT_test_ds)
y_test = ENT_datasets_dict['test']['label']
# convert to tensorflow dataset
secondary_test_ds = tf.data.Dataset.from_tensor_slices(({'entity_hidden_state_layer': test_ENTITY_hidden_state_np, 'cls_token_LHS_layer': test_CLS_token_ouput_np}, y_test)).batch(32)

16/16 [==============================] - 0s 5ms/step


### [CLS] & Entity - Position-wise FFN

In [50]:
def SEPARATED_both_FFN_concat():

    entity_hidden_state_layer = tf.keras.layers.Input(shape=(768,), dtype=tf.float64, name='entity_hidden_state_layer')
    cls_token_LHS_layer = tf.keras.layers.Input(shape=(768,), dtype=tf.float64, name='cls_token_LHS_layer')

    # Entity token passes through FFN after self-attention in DeBERTa, so it is not repeated here
    ff_entity1 = tf.keras.layers.Dense(units=3072, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="gelu",
                                        name="ff_hidden1")(entity_hidden_state_layer)
    ff_entity2 = tf.keras.layers.Dense(units=768, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="gelu",
                                        name="ff_hidden2")(ff_entity1)


    ff_cls1 = tf.keras.layers.Dense(units=3072, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="gelu",
                                        name="ff_cls1")(cls_token_LHS_layer)
    ff_cls2 = tf.keras.layers.Dense(units=768, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="gelu",
                                        name="ff_cls2")(ff_cls1)

    concat_layer = tf.keras.layers.concatenate([ff_cls2, ff_entity2], name="concat")

    pooler_layer = tf.keras.layers.Dense(units=768, 
                                        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                        activation="tanh",
                                        name="dense")(concat_layer)

    cls_dropout = tf.keras.layers.Dropout(0.1, name='cls_dropout')(pooler_layer)
    classifier = tf.keras.layers.Dense(units=1,
                                    kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                    name="classifier")(cls_dropout)

    model = tf.keras.models.Model(inputs=[entity_hidden_state_layer, cls_token_LHS_layer], outputs=classifier)
    return model

In [ ]:
name = 'SEPARATED_both_FFN_concat'
ds_type="masked_extra_field"
fit_model(name, SEPARATED_both_FFN_concat, secondary_train_ds, secondary_val_ds)

### [CLS] & Entity - CNN-BiLSTM

In [63]:
def SEPARATED_both_cnn_bilstm():
    entity_hidden_state_layer = tf.keras.layers.Input(shape=(768,), dtype=tf.float64, name='entity_hidden_state_layer')
    cls_token_LHS_layer = tf.keras.layers.Input(shape=(768,), dtype=tf.float64, name='cls_token_LHS_layer')
    
    # add pooling and dense layers to combine information from both inputs
    cls_pooler_layer = tf.keras.layers.Dense(units=768, activation="tanh", name="cls_pooler")(cls_token_LHS_layer)
    entity_pooler_layer = tf.keras.layers.Dense(units=768, activation="tanh", name="entity_pooler")(entity_hidden_state_layer)
    
    combined_pooled_hidden_states = tf.keras.layers.concatenate([cls_pooler_layer, entity_pooler_layer], axis=1)
    
    # add a CNN-BiLSTM layer
    cnn_lstm = tf.keras.Sequential()
    cnn_lstm.add(tf.keras.layers.Reshape((1, combined_pooled_hidden_states.shape[1])))
    cnn_lstm.add(tf.keras.layers.Conv1D(128, kernel_size=3, padding='same', activation='relu'))
    cnn_lstm.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
    cnn_lstm_output = cnn_lstm(combined_pooled_hidden_states)

    cls_dropout = tf.keras.layers.Dropout(0.1, name='cls_dropout')(cnn_lstm_output)
    classifier = tf.keras.layers.Dense(units=1,
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=config.initializer_range),
                                       name="classifier")(cls_dropout)
    
    model = tf.keras.models.Model(inputs=[entity_hidden_state_layer, cls_token_LHS_layer], outputs=classifier)
    return model


In [ ]:
name = 'SEPARATED_both_cnn_bilstm'
ds_type="masked_extra_field"
fit_model(name, SEPARATED_both_cnn_bilstm, secondary_train_ds, secondary_val_ds)

### [CLS] & Entity - Regression HPOpt

In [76]:
# --TRAIN--
ENTITY_hidden_state_np, CLS_token_ouput_np = create_arrays('train', CLS_train_ds, ENT_train_ds)
# Combine
arr_concat = np.concatenate([ENTITY_hidden_state_np, CLS_token_ouput_np], axis=1)
concat_df_train = pd.DataFrame(arr_concat)
# Create df
concat_df_train[concat_df_train.columns[-1]+1] = ENT_datasets_dict['train']['label'] # <------- HERE
concat_df_train[concat_df_train.columns[-1]] = concat_df_train[concat_df_train.columns[-1]].round(3)
# Split into X_train, y_train
X_train = concat_df_train.loc[:, concat_df_train.columns[:-1]]
y_train = concat_df_train.loc[:, concat_df_train.columns[-1]]


# --VAL--
ENTITY_hidden_state_np, CLS_token_ouput_np = create_arrays('val', CLS_val_ds, ENT_val_ds)
# Combine
arr_concat = np.concatenate([ENTITY_hidden_state_np, CLS_token_ouput_np], axis=1)
concat_df_val = pd.DataFrame(arr_concat)
# Create df
concat_df_val[concat_df_val.columns[-1]+1] = ENT_datasets_dict['val']['label'] # <------- HERE
concat_df_val[concat_df_val.columns[-1]] = concat_df_val[concat_df_val.columns[-1]].round(3)
# Split into X_val, y_val
X_val = concat_df_val.loc[:, concat_df_val.columns[:-1]]
y_val = concat_df_val.loc[:, concat_df_val.columns[-1]]


# --TEST--
ENTITY_hidden_state_np, CLS_token_ouput_np = create_arrays('test', CLS_test_ds, ENT_test_ds)
# Combine
arr_concat = np.concatenate([ENTITY_hidden_state_np, CLS_token_ouput_np], axis=1)
concat_df_test = pd.DataFrame(arr_concat)
# Create df
concat_df_test[concat_df_test.columns[-1]+1] = ENT_datasets_dict['test']['label'] # <------- HERE
concat_df_test[concat_df_test.columns[-1]] = concat_df_test[concat_df_test.columns[-1]].round(3)
# Split into X_test, y_test
X_test = concat_df_test.loc[:, concat_df_test.columns[:-1]]
y_test = concat_df_test.loc[:, concat_df_test.columns[-1]]

16/16 [==============================] - 0s 5ms/step


In [ ]:
# define search
model = HyperoptEstimator(regressor=any_regressor('reg'), preprocessing=any_preprocessing('pre'), loss_fn=mean_squared_error, algo=tpe.suggest, max_evals=150, trial_timeout=30)
# perform the search
model.fit(X_train, y_train)
# summarize performance
mae = model.score(X_val, y_val)
print("MAE: %.3f" % mae)
# summarize the best model
print(model.best_model())

100%|██████████| 150/150 [00:00<00:00,  1.70trial/s, best loss: 0.005557062318529729]
MAE: 0.739
{'learner': SVR(C=1.0907584875271739, coef0=0.34641173684893933, degree=1,
    epsilon=0.09778996627429437, kernel='poly', shrinking=False,
    tol=2.8433823625156128e-05), 'preprocs': (StandardScaler(with_mean=False, with_std=False),), 'ex_preprocs': ()}


In [75]:
# fit SVR model to X_train and y_train
def fit_svr(X_train, y_train):
    svr = SVR(C=1.0907584875271739, coef0=0.34641173684893933, degree=1,epsilon=0.09778996627429437, kernel='poly', shrinking=False,tol=2.8433823625156128e-05)
    svr.fit(X_train, y_train)
    return svr

# predict using SVR model   
def predict_svr(svr, X_test_or_val):
    y_pred = svr.predict(X_test_or_val)
    return y_pred

In [78]:
svr = fit_svr(X_train, y_train)

In [79]:
y_val_pred = predict_svr(svr, X_val)
cosine_score(y_val, y_val_pred)

0.8605199448729892

# Evaluate All Models on Test Set

In [82]:
model_dict = {
                    'cls_baseline' : [],
                    'cls_tgt_replaced_baseline' : [],
                    'entity_baseline' : [],
              
                    'ATTACHED_both_FFN_concat' : [],
                    'ATTACHED_both_cnn_bilstm' : [],

                    'SEPARATED_both_cnn_bilstm' : [],
                    'SEPARATED_both_FFN_concat' : [],
}

for name, ds_type in [('cls_baseline', 'raw'),
                      ("cls_tgt_replaced_baseline", 'masked'),
                      ("entity_baseline", 'masked_extra_field'),

                      ('ATTACHED_both_FFN_concat', 'masked_extra_field'),
                      ("ATTACHED_both_cnn_bilstm", 'masked_extra_field'),

                      ('SEPARATED_both_FFN_concat', 'masked_extra_field'),
                      ("SEPARATED_both_cnn_bilstm", 'masked_extra_field'),
                      
                      
                      ]:
    
    model = keras.models.load_model(f'/content/drive/MyDrive/ColabCode/MLE/best_models/{name}.h5',
                                    custom_objects={"TFDebertaV2Model": TFDebertaV2Model, "TFDebertaV2StableDropout":TFDebertaV2StableDropout})
    
    max_seq, datasets_dict, train_ds, val_ds, test_ds = create_tokenized_tf_datasets_caller(ds_type, batch_size=32)

    if name[:10]=='SEPARATED_':
        predicted_sentiments = model.predict(secondary_test_ds)
    else:
        predicted_sentiments = model.predict(test_ds)
    y_pred = predicted_sentiments.flatten()

    # Flatten the test labels and predicted sentiments into 1D arrays
    y_true = np.concatenate([y for x, y in test_ds], axis=0)

    # Calculate cosine similarity and MSE
    result = cosine_score(y_pred, y_true)
    loss = mean_squared_error(y_true, y_pred)

    model_dict[name] = [loss, result]

y_test_pred = predict_svr(svr, X_test)
model_dict['SEPARATED_both_SVR'] = [mean_squared_error(y_true, y_test_pred), cosine_score(y_test, y_test_pred)]

output.clear()

model_dict

{'cls_baseline': [0.047548804, 0.8566936254501343],
 'cls_tgt_replaced_baseline': [0.049198974, 0.8538933992385864],
 'entity_baseline': [0.041516673, 0.8736073970794678],
 'ATTACHED_both_FFN_concat': [0.04434816, 0.8661375045776367],
 'ATTACHED_both_cnn_bilstm': [0.055714294, 0.8492858409881592],
 'SEPARATED_both_cnn_bilstm': [0.03891518, 0.8799764513969421],
 'SEPARATED_both_FFN_concat': [0.038986906, 0.8794136643409729],
 'SEPARATED_both_SVR': [0.03990358684206234, 0.8762612325942858]}

In [83]:
df = pd.DataFrame.from_dict(model_dict, orient='index', columns=['MSE', 'cosine similarity'])
df = df.apply(lambda x: [round(i, -int(math.floor(math.log10(abs(i))))+2) for i in x])
df

,MSE,cosine similarity
cls_baseline,0.0475,0.857
cls_tgt_replaced_baseline,0.0492,0.854
entity_baseline,0.0415,0.874
ATTACHED_both_FFN_concat,0.0443,0.866
ATTACHED_both_cnn_bilstm,0.0557,0.849
SEPARATED_both_cnn_bilstm,0.0389,0.880
SEPARATED_both_FFN_concat,0.0390,0.879
SEPARATED_both_SVR,0.0399,0.876
